In [1]:
#@title Setup Environment
from IPython.display import clear_output, display
!pip install deepface
!pip install mtcnn

import os
import json
from deepface import DeepFace
from deepface.commons import functions
from mtcnn.mtcnn import MTCNN
clear_output()

In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
images_path = "/content/imgs" #@param {type:"string"}

def output_img_atts(img_atts: list, image_path: str):
    if not os.path.isdir(image_path):
        image_path = image_path.rsplit("/", 1)[0]
    output_file_path = os.path.join(image_path, "face_analysis.json")
    with open(output_file_path, 'w') as f:
        json.dump(img_atts, f)

def analyze_faces(image_path: str):
    img_exts = ["png", "jpg", "jpeg"]
    img_atts = []
    actions = ['age', 'gender', 'race', 'emotion']
    detector_backend = 'mtcnn'

    if os.path.isdir(image_path):
        for root, dirs, files in os.walk(image_path):
            images = [f for f in files if f.split(".")[-1].lower() in img_exts]
            images = [os.path.join(root, i) for i in images]

            for image_path in images:
                try:
                    faces = functions.extract_faces(image_path, detector_backend=detector_backend)
                    if len(faces) > 0:
                        face_img = faces[0]  # Assuming only one face per image
                        if face_img.shape[0] > 0 and face_img.shape[1] > 0:
                            objs = DeepFace.analyze(face_img, actions=actions, enforce_detection=False)
                            for i in range(len(objs)):
                                objs[i]["image_path"] = image_path
                            img_atts.extend(objs)
                            print(f"Image: {image_path}")
                            print(f"Attributes: {objs}")
                            print("\n")
                except Exception as e:
                    print(f"Error processing image: {image_path}")
                    print(f"Error: {e}")
                    print("\n")
    else:
        try:
            faces = functions.extract_faces(image_path, detector_backend=detector_backend)
            if len(faces) > 0:
                face_img = faces[0]  # Assuming only one face per image
                if face_img.shape[0] > 0 and face_img.shape[1] > 0:
                    objs = DeepFace.analyze(face_img, actions=actions, enforce_detection=False)
                    for i in range(len(objs)):
                        objs[i]["image_path"] = image_path
                    img_atts.extend(objs)
                    print(f"Image: {image_path}")
                    print(f"Attributes: {objs}")
                    print("\n")
        except Exception as e:
            print(f"Error processing image: {image_path}")
            print(f"Error: {e}")
            print("\n")

    output_img_atts(img_atts, image_path)
    return img_atts

img_atts = analyze_faces(images_path)




In [4]:
images_path = "/content/drive/MyDrive/AI/CURATED_DATASETS/FFHQ/test" #@param {type:"string"}
actions = ['age', 'gender',' race', 'emotion'] #@param {type:"raw"}
detector_backend = "retinaface" #@param ["opencv", "retinaface", "mtcnn", "ssd", "dlip", "mediapipe"]
if not detector_backend:
  detector_backend = "opencv" 
IMG_EXTS = ["png", "jpg", "jpeg"]

def get_image_paths(image_path: str) -> list:
    if os.path.isdir(image_path):
        images = []
        for root, dirs, files in os.walk(image_path):
            for file_name in files:
                if file_name.split(".")[-1].lower() in IMG_EXTS:
                    image_path = os.path.join(root, file_name)
                    images.append(image_path)
    else:
        images = [image_path]
    return images

def output_img_atts(img_atts: list, image_path: str):
    if not os.path.isdir(image_path):
        image_path = image_path.rsplit("/", 1)[0]
    output_file_path = os.path.join(image_path, "face_analysis.json")
    with open(output_file_path, 'w') as f:
        json.dump(img_atts, f)

def analyze_faces(input_image_path: str, actions: list, detector_backend: str):
    img_atts = []
    images = get_image_paths(input_image_path)

    for image_path in images:
        try:
            objs = DeepFace.analyze(image_path, actions=actions, enforce_detection=True)
        except Exception as e:
            print(f"No face detected for {image_path}")
            objs = [{"face_detected": False}]
      
        for i in range(len(objs)):
            objs[i]["image_path"] = image_path
            
        img_atts.extend(objs)

    output_img_atts(img_atts, input_image_path)
    return img_atts

img_atts = analyze_faces(images_path, actions, detector_backend)




Action: emotion: 100%|██████████| 4/4 [00:00<00:00, 10.21it/s]
